# **Multy commodity flow problem**

 ***MIP Model***

**Sets and Indices**

- **I**: Set of source nodes
- **J**: Set of destination nodes
- **K**: Set of commodities

**Parameters**

- $c_{ijk}$: Cost of transporting one unit of commodity *k* from source *i* to destination *j*
- $f_{ijk}$: Fixed cost for using the route from source *i* to destination *j* for commodity *k*
- $r_{ijk}$: Revenue generated from transporting one unit of commodity *k* from source *i* to destination *j*
- $d_{jk}$: Demand for commodity *k* at destination *j*
- $s_{ik}$: Supply of commodity *k* at source *i*
- $M$: A large constant (big-M) to ensure constraints work correctly

**Decision variables**

- $x_{ijk}$: Amount of commodity *k* transported from source *i* to destination *j*
- $v_{jk}$: Amount of demand served for commodity *k* at destination *j*
- $y_{ijk}$: Binary variable, equals 1 if any commodity *k* is transported from source *i* to destination *j*, 0 otherwise

**Objective function**

1. Minimize Costs Associated with Flow
$$
\text{Minimize } Z_1 = \sum_{i \in I} \sum_{j \in J} \sum_{k \in K} (c_{ijk} x_{ijk} + f_{ijk} y_{ijk})
$$

2. Maximize Revenues
$$
\text{Maximize } Z_2 = \sum_{i \in I} \sum_{j \in J} \sum_{k \in K} r_{ijk} x_{ijk} y_{ijk}
$$

3. Maximize Demand Served
$$
\text{Maximize } Z_3 = \sum_{j \in J} \sum_{k \in K} v_{jk}
$$

4. Minimize Spillage

$$
\text{Minimize } Z_4 = \sum_{j \in J} \sum_{k \in K} (d_{jk} - \sum_{i \in I} x_{ijk} y_{ijk})
$$


**Constraints**

1. Flow Conservation at Source Nodes
$$
\sum_{j \in J} x_{ijk} \leq s_{ik} \quad \forall i \in I, \forall k \in K
$$

2. Flow Conservation at Destination Nodes
$$
\sum_{i \in I} x_{ijk} + (d_{jk} - \sum_{i \in I} x_{ijk}) = d_{jk} \quad \forall j \in J, \forall k \in K
$$

3. Demand Served Constraint
$$
v_{jk} = \sum_{i \in I} x_{ijk} y_{ijk} \quad \forall j \in J, \forall k \in K
$$

4. Route Usage Constraint
$$
x_{ijk} \leq M \cdot y_{ijk} \quad \forall i \in I, \forall j \in J, \forall k \in K
$$



**Full Model**

$$
\begin{align*}
\text{Minimize } & Z = w_1 \sum_{i \in I} \sum_{j \in J} \sum_{k \in K} (c_{ijk} x_{ijk} + f_{ijk} y_{ijk}) - w_2 \sum_{i \in I} \sum_{j \in J} \sum_{k \in K} r_{ijk} x_{ijk} y_{ijk} - w_3 \sum_{j \in J} \sum_{k \in K} v_{jk} + w_4 \sum_{j \in J} \sum_{k \in K} (d_{jk} - \sum_{i \in I} x_{ijk} y_{ijk}) \\
\text{subject to} \quad & \sum_{j \in J} x_{ijk} \leq s_{ik} \quad \forall i \in I, \forall k \in K \\
& \sum_{i \in I} x_{ijk} = v_{jk} \quad \forall j \in J, \forall k \in K \\
& v_{jk} = \sum_{i \in I} x_{ijk} y_{ijk} \quad \forall j \in J, \forall k \in K \\
& x_{ijk} \leq M \cdot y_{ijk} \quad \forall i \in I, \forall j \in J, \forall k \in K \\
& x_{ijk} \geq 0 \quad \forall i \in I, \forall j \in J, \forall k \in K \\
& v_{jk} \geq 0 \quad \forall j \in J, \forall k in K \\
& y_{ijk} \in \{0, 1\} \quad \forall i \in I, \forall j \in J, \forall k \in K
\end{align*}
$$

**code**

In [3]:
import pulp
class MultyCommodityFlow:
    def __init__(self, I,J,K,c,f,r,d,s,M,w1,w2,w3,w4):
        # Sets
        self.I = I  # Set of source nodes
        self.J = J  # Set of destination nodes
        self.K = K  # Set of commodities
        
        # Parameters
        self.c = c  # Cost of transporting one unit of commodity k from source i to destination j
        self.f = f  # Fixed cost for using the route from source i to destination j for commodity k
        self.r = r  # Revenue generated from transporting one unit of commodity k from source i to destination j
        self.d = d  # Demand for commodity k at destination j
        self.s = s  # Supply of commodity k at source i
        self.M = M
        self.w1 = w1  # Weight for the cost objective
        self.w2 = w2  # Weight for the revenue objective
        self.w3 = w3  # Weight for the demand served objective
        self.w4 = w4  # Weight for the spillage objective

        #initialize the problem
        self.problem = pulp.LpProblem("MultyCommodityFlow", pulp.LpMinimize)

        #decition variables
        self.x = pulp.LpVariable.dicts("x", [(i,j,k) for i in self.I for j in self.J for k in self.K], lowBound=0, cat = pulp.LpContinuous)
        self.y = pulp.LpVariable.dicts("y", [(i,j,k) for i in self.I for j in self.J for k in self.K], cat = pulp.LpBinary)
        self.v = pulp.LpVariable.dicts("v", [(j,k) for j in self.J for k in self.K], lowBound=0, cat = pulp.LpContinuous)
        self.z = pulp.LpVariable.dicts("z", [(i, j, k) for i in self.I for j in self.J for k in self.K], lowBound=0, cat=pulp.LpContinuous)

    def build_model(self):
        #objective func
        self.problem += (
            self.w1 * (
                pulp.lpSum(self.c[i, j, k] * self.x[i, j, k] for i in self.I for j in self.J for k in self.K) 
                - pulp.lpSum(self.f[i, j, k] * self.y[i, j, k] for i in self.I for j in self.J for k in self.K)
            )
            - self.w2 * pulp.lpSum(self.r[i, j, k] * self.z[i, j, k] for i in self.I for j in self.J for k in self.K)
            - self.w3 * pulp.lpSum(self.v[j, k] for j in self.J for k in self.K)
            + self.w4 * (
                pulp.lpSum(self.d[j, k] for j in self.J for k in self.K)
                - pulp.lpSum(self.z[i, j, k] for i in self.I for j in self.J for k in self.K)
            )
        )
        
        #contraints
        #1. Flow conservation at source nodes
        for i in self.I:
            for k in self.K:
                self.problem += pulp.lpSum(self.x[i,j,k] for j in self.J) <= self.s[i,k]
        #2. Flow conservation at destination nodes
        for j in self.J:
            for k in self.K:
                self.problem += pulp.lpSum(self.x[i,j,k] for i in self.I) == self.v[j,k]
        
        # 3. Definition of z (auxiliary variable for product of x and y)
        for i in self.I:
            for j in self.J:
                for k in self.K:
                    self.problem += self.z[i, j, k] <= self.M * self.y[i, j, k]  
                    self.problem += self.z[i, j, k] <= self.x[i, j, k]  
                    self.problem += self.z[i, j, k] >= self.x[i, j, k] - self.M * (1 - self.y[i, j, k])  
        #4. Linking x and y with big M
        for i in self.I:
            for j in self.J:
                for k in self.K:
                    self.problem += self.x[i,j,k] <= self.M * self.y[i,j,k]
        # 5. Definition of v (amount of demand served at each destination)
        for j in self.J:
            for k in self.K:
                self.problem += self.v[j, k] == pulp.lpSum(self.x[i, j, k] for i in self.I)
    
    def solve(self):
        # Solve the problem
        self.problem.solve(pulp.CPLEX_PY(msg=True))
        
        # Print the status of the solution
        status = pulp.LpStatus[self.problem.status]
        print("\nObjective Value:", pulp.value(self.problem.objective))
        print("Status:", status)
        
        # If the solution is feasible, print the values of the decision variables
        if status == 'Optimal' or status == 'Feasible':
            print("\nDecision Variables:")
            # for v in self.problem.variables():
            #     print(v.name, "=", v.varValue)
        else:
            print("No feasible solution found.")

In [5]:
#example
# Define sets
I = ["S1", "S2", "S3"]
J = ["D1", "D2"]
K = ["C1", "C2"]

# Define parameters
c = {("S1", "D1", "C1"): 2, ("S1", "D1", "C2"): 3, ("S1", "D2", "C1"): 4, ("S1", "D2", "C2"): 1,
     ("S2", "D1", "C1"): 3, ("S2", "D1", "C2"): 2, ("S2", "D2", "C1"): 1, ("S2", "D2", "C2"): 4,
     ("S3", "D1", "C1"): 5, ("S3", "D1", "C2"): 6, ("S3", "D2", "C1"): 2, ("S3", "D2", "C2"): 3}
f = {("S1", "D1", "C1"): 0, ("S1", "D1", "C2"): 0, ("S1", "D2", "C1"): 0, ("S1", "D2", "C2"): 0,
     ("S2", "D1", "C1"): 0, ("S2", "D1", "C2"): 0, ("S2", "D2", "C1"): 0, ("S2", "D2", "C2"): 0,
     ("S3", "D1", "C1"): 0, ("S3", "D1", "C2"): 0, ("S3", "D2", "C1"): 0, ("S3", "D2", "C2"): 0}
r = {("S1", "D1", "C1"): 0, ("S1", "D1", "C2"): 0, ("S1", "D2", "C1"): 0, ("S1", "D2", "C2"): 0,
     ("S2", "D1", "C1"): 0, ("S2", "D1", "C2"): 0, ("S2", "D2", "C1"): 0, ("S2", "D2", "C2"): 0,
     ("S3", "D1", "C1"): 0, ("S3", "D1", "C2"): 0, ("S3", "D2", "C1"): 0, ("S3", "D2", "C2"): 0}
d = {("D1", "C1"): 10, ("D1", "C2"): 15, ("D2", "C1"): 20, ("D2", "C2"): 25}
s = {("S1", "C1"): 20, ("S1", "C2"): 25, ("S2", "C1"): 30, ("S2", "C2"): 35, ("S3", "C1"): 40, ("S3", "C2"): 45}

# Define constants
M = 1000
w1 = 1
w2 = 0
w3 = 0
w4 = 0

# Create an instance of the MultyCommodityFlow class
model = MultyCommodityFlow(I, J, K, c, f, r, d, s, M, w1, w2, w3, w4)

# Build the model
model.build_model()

# Solve the problem
model.solve()

Version identifier: 22.1.1.0 | 2023-06-15 | d64d5bd77
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.00 ticks)
Parallel b&c, 16 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.00 ticks)
Cplex status= 101

Objective Value: 0.0
Status: Optimal

Decision Variables:
